In [1]:
import pandas as pd


In [4]:
!ls ../data

AI4Code.zip	       test   train_ancestors.csv
sample_submission.csv  train  train_orders.csv


In [10]:
orders=pd.read_csv("../data/train_orders.csv")
ancestors=pd.read_csv("../data/train_ancestors.csv")
sample_submission=pd.read_csv("../data/sample_submission.csv")

In [3]:
!ls ../data/train

00001756c60be8.json  558972144d2240.json  aa5d2a55a4b73d.json
00015c83e2717b.json  558a6246a732b4.json  aa5dd4dc05b770.json
0001bdd4021779.json  558aca3c520788.json  aa5dee2e1510a8.json
0001daf4c2c76d.json  558af7973e062f.json  aa5e6885078bb6.json
0002115f48f982.json  558b4a72637134.json  aa5f25e7574d65.json
00035108e64677.json  558b52aca530ef.json  aa5f34b08b044e.json
00038c2941faa0.json  558b9560830081.json  aa60476c559154.json
00039fc77a9dd1.json  558c1c9550d2e3.json  aa6050b56de66e.json
00058dc97eae4b.json  558c3d609123e5.json  aa608b05c417c5.json
000597ac4c6700.json  558cb5d9e8bc5e.json  aa6094b917d527.json
000624d747afd3.json  558cf59e9d6158.json  aa61a6fb5a2419.json
00062ab8487156.json  558de6a009749f.json  aa61b0ed1d4c81.json
000757b90aaca0.json  558de9ace80eb8.json  aa61bd9328d222.json
0007f21ee357b5.json  558fc19d7c28b4.json  aa61d69b65bf42.json
000890decea38e.json  558fce17a4b5f1.json  aa627454939094.json
0008ba887b3817.json  558fd02cfb6356.json  aa62c02ca6639b.json
0009acaa

05a807b7895154.json  5af249257d009f.json  afec26662e5291.json
05a83d5729a02a.json  5af26a4be60767.json  afec95853c34ae.json
05a8d231485473.json  5af290f6732724.json  afed384ac689cf.json
05a8f0ec1b727b.json  5af30ec8f521f4.json  afee064777d8cd.json
05a903a875b9b9.json  5af3fcb78631fa.json  afee2f2e3c2b8f.json
05a98249b2d66a.json  5af4afc7082c90.json  afeeb309ec2ba5.json
05a98e8af42f31.json  5af5e4a7acbd1b.json  afef6ca9e069f9.json
05a9912c29b606.json  5af68248f56925.json  afef98803cc846.json
05a99ffd93f973.json  5af698bfc77cc5.json  afefa15ed2d63e.json
05aa9fd68de1ad.json  5af6e08ab8250b.json  aff04a030c5440.json
05aaa66e6ea359.json  5af73cf389890d.json  aff062701a4ad7.json
05aab46ea90a11.json  5af7d8c60de04a.json  aff0cb8a2f096e.json
05aad46ab9191d.json  5af836821b8f3e.json  aff0cba56d0fed.json
05ab5ec578f906.json  5af85637f92176.json  aff0f839b99c61.json
05abf523254a14.json  5af89b641a81ee.json  aff1452b2440e6.json
05ac4381a5cb97.json  5af9003a6d84bc.json  aff300dda2bd1d.json
05adac71

0a3dea1dab4c64.json  5f97e2cd0b4ce4.json  b4b2ef5958bb1c.json
0a3dfc4fe25373.json  5f982e97642c1c.json  b4b422872a6106.json
0a3e841e5c91fe.json  5f98381f7e5d67.json  b4b4ba1c91611a.json
0a3ea33c4f7f61.json  5f983a47ed8699.json  b4b51e18f2612b.json
0a3fb691838d29.json  5f99885ed355c7.json  b4b5c67837818c.json
0a3fd183387e42.json  5f99e45ee6347d.json  b4b6dc8933d29d.json
0a3fd67dd2acfa.json  5f99fead885618.json  b4b6ed85f6b456.json
0a3ffe517b4ca5.json  5f9a22bab0e0f0.json  b4b706dffc0ba4.json
0a40117fdcc493.json  5f9a5dda0ecdb2.json  b4b7473e49d3ee.json
0a411867331bd4.json  5f9aacaf396720.json  b4b8581b10f648.json
0a4144cd445b09.json  5f9ab4e359c957.json  b4b869daceb1af.json
0a41bd802f5c9e.json  5f9b3301f0d31f.json  b4b88aa22a127a.json
0a41d46f302d64.json  5f9b7396cba7b6.json  b4b89cd274fe06.json
0a425fc5ddf5ff.json  5f9c6071e2d08a.json  b4b95f1b19695c.json
0a42640d9edee3.json  5f9cd0110f245d.json  b4b9928afe75fe.json
0a46307ec6f0af.json  5f9e1b08f05378.json  b4b9d1c748ee7d.json
0a472ed7

1a40681d7c85f8.json  7024ad21af8de4.json  c4ff494af72535.json
1a40b11ce2bdf5.json  702579b4ee9810.json  c5001b9d8b5786.json
1a410fd63162a9.json  702612103d5a0a.json  c500bf16836add.json
1a420b7e49e5b9.json  70261b8c6e4b33.json  c500d0989ebfd8.json
1a4267053ed070.json  7026838b0395eb.json  c501dae6a55efa.json
1a4267160eb2c6.json  7027330771b517.json  c502783f064858.json
1a428f39b4343b.json  7027de101e1e18.json  c5028e1c29f194.json
1a429a2c80c712.json  7028c077c8c0a9.json  c5031533408098.json
1a42ce3f219ac9.json  70295271c77b9e.json  c503b6d6b3af8e.json
1a431351e6b73a.json  702b046741e6ce.json  c504d8957fed6b.json
1a43af4d0beeb3.json  702b58c88ae72a.json  c505ca06ba3859.json
1a4509150b622f.json  702ba261f34ad7.json  c5069873f00b24.json
1a462660fee7fa.json  702c170b707bd7.json  c506effa6c63f0.json
1a4670bcaa5aac.json  702c1b2158d113.json  c5074231139287.json
1a4698816719db.json  702c3356d8e748.json  c509de2d3d1e80.json
1a46fbc35d91a8.json  702c5a7a78e344.json  c509e44f07a925.json
1a470493

29859bbdf012eb.json  7f012b775d7e01.json  d42d35da73c609.json
29863696623563.json  7f024fe8d8b8c6.json  d42dbc336a4bef.json
2986fe491e9c86.json  7f02bd81ff71a9.json  d42dc04bf6ae38.json
29873bc18b336d.json  7f033f69354e3e.json  d42ec94ef39775.json
2987453465ac59.json  7f0464cc414b0d.json  d42fa1dd955ac4.json
298773f06cb4f6.json  7f0499b83309d1.json  d42fa5a4564566.json
2987b5b0013d92.json  7f04d0a4658c55.json  d42faf58e5f8fd.json
2987d26460e71a.json  7f05288aa346f2.json  d43052d3bfa994.json
29884bdb8e8f8d.json  7f052aacab5cca.json  d430e779e6f614.json
2989c48ead6861.json  7f05317b649631.json  d430ef9dd89af5.json
298aa171b7747c.json  7f076acadd578e.json  d4319624e7b384.json
298b16f8097ec6.json  7f080f90580dbb.json  d43224aee0a698.json
298b43196d9019.json  7f089c23e20be8.json  d432d2e822ac50.json
298c825517aa05.json  7f09786e47d1f2.json  d433fbb4816291.json
298c9f7d960f7b.json  7f0999c04d314a.json  d4340b2aec648c.json
298e90edc2939b.json  7f09a05b5d33dd.json  d434e132d5cd3a.json
298ef184

367f4bf79f2b95.json  8bdb7319f06dc7.json  e13e1ac313376d.json
367fa4def3b55a.json  8bdb9a28c14613.json  e13e99e3c535c6.json
367fbc214e0115.json  8bdcafee69b1a4.json  e13f9dedaa6a79.json
36800f25fe4da2.json  8bdd283b8459c4.json  e13fad283959e8.json
3680344a3fb678.json  8bddc0358563ab.json  e1404bf65831ea.json
36808b8dc7eb6d.json  8bde39710e0977.json  e1405e1d5b5419.json
36815e4226c2a0.json  8bde68f33000ef.json  e140acb75b2a52.json
36815ea72e1cc6.json  8bdf7c4ba7a997.json  e1411fb7fa4e22.json
3681e26f71a5f0.json  8bdfbe2fd7972f.json  e1424be7005bf9.json
3682529827af6b.json  8bdfd72369c8c3.json  e142f60f9ad880.json
36827a45ca8f0b.json  8be00514fe0295.json  e14326c27b0adc.json
3682dd142fc90d.json  8be066ceb4d55c.json  e1438f2a2a6d5e.json
36834d87ae312d.json  8be222c16f6c98.json  e1439462ec59b2.json
3683c1daf490d2.json  8be2a32bbf8a02.json  e1449f7adfc5c3.json
36840ca280e777.json  8be3546e41d202.json  e14512745657e9.json
36846dd9e02ad3.json  8be3944efc629d.json  e1465f98f459d9.json
3684d471

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
import json
with open("../data/train/00035108e64677.json", "r") as f:
    x=json.loads(f.read())

In [6]:
x

{'cell_type': {'2fa1f27b': 'code',
  'f3c2de19': 'code',
  '56639d19': 'code',
  '13362913': 'code',
  'ab817107': 'code',
  '6092fe8c': 'code',
  'a0e5ace3': 'code',
  '8111e543': 'code',
  '1d07ce06': 'code',
  '32f1cff2': 'code',
  '977d9ffd': 'code',
  '697468b0': 'code',
  'bf152f54': 'code',
  '13abf70f': 'code',
  '67a91c07': 'code',
  '1e7877e9': 'code',
  'ac3558e5': 'code',
  '25ec8004': 'code',
  'fc04571f': 'code',
  '9e2306cf': 'code',
  '6382fc3c': 'code',
  '43903da9': 'code',
  '977285e4': 'code',
  'd4a278de': 'markdown',
  '84f836ca': 'markdown',
  '3496fbfe': 'markdown',
  'd75feb42': 'markdown',
  'aabc28d8': 'markdown',
  '214d8ef9': 'markdown',
  '395866fa': 'markdown',
  'e470479f': 'markdown',
  '5f5145c1': 'markdown',
  'f9a24fbd': 'markdown',
  '719854c4': 'markdown'},
 'source': {'2fa1f27b': '# Basic Libraries\nimport numpy as np\nimport pandas as pd\nimport seaborn as sb\nimport matplotlib.pyplot as plt # we only need pyplot\nsb.set() # set the default Seabo

In [11]:
sample_submission

,id,cell_order
0,0009d135ece78d,ddfd239c c6cd22db 1372ae9b 90ed07ab 7f388a41 2...
1,0010483c12ba9b,54c7cab3 fe66203e 7844d5f8 5ce8863c 4a0777c4 4...
2,0010a919d60e4f,aafc3d23 80e077ec b190ebb4 ed415c3c 322850af c...
3,0028856e09c5b7,012c9d02 d22526d1 3ae7ece3 eb293dfc


In [15]:
!ls -lh ../data/test

total 60K
-rw-rw-r-- 1 andreas andreas 4.0K May  9 15:31 0009d135ece78d.json
-rw-rw-r-- 1 andreas andreas  23K May  9 15:31 0010483c12ba9b.json
-rw-rw-r-- 1 andreas andreas  23K May  9 15:31 0010a919d60e4f.json
-rw-rw-r-- 1 andreas andreas 4.9K May  9 15:31 0028856e09c5b7.json


In [18]:
import os 

test_data={}

for filename in os.listdir("../data/test"):
    print(filename)
    with open("../data/test/"+filename, "r") as f:
        test_data[filename.split(".")[0]]=json.loads(f.read())

0028856e09c5b7.json
0010483c12ba9b.json
0009d135ece78d.json
0010a919d60e4f.json


In [37]:
import random

s=pd.DataFrame(list(map(lambda key: {"id": key, 
        "cell_order": " ".join(random.sample(list(test_data[key]["cell_type"].keys()), len(test_data[key]["cell_type"])))}, 
        test_data.keys())))

In [38]:
submission_name="random"
os.makedirs("../submissions/"+submission_name)
s.to_csv("../submissions/"+submission_name+"/submission.csv", index=False)

In [40]:
!ls ../submissions/random

submission.csv


In [46]:
!kaggle competitions submit -c AI4Code -f ../submissions/random/submission.csv -m "First random submission"

100%|████████████████████████████████████████████| 875/875 [00:02<00:00, 305B/s]
400 - Bad Request


In [47]:
!cat ../submissions/random/submission.csv

id,cell_order
0028856e09c5b7,eb293dfc 012c9d02 3ae7ece3 d22526d1
0010483c12ba9b,54c7cab3 5ce8863c fe66203e 4703bb6d 02a0be6d 7844d5f8 4a0777c4 7f270e34 4a32c095 865ad516
0009d135ece78d,0a226b6a ba55e576 90ed07ab 39e937ec 1372ae9b ddfd239c 06dbf8cf 2843a25a c6cd22db 8cb8d28a 7f388a41 f9893819 e25aa9bd
0010a919d60e4f,0e2529e8 21b6fb8f 3741e756 4ae17669 bc8eaa53 bac960d3 ef01da10 4356ab34 0115f7f5 aafc3d23 18ce8cc0 23607d04 1d4dbeae b190ebb4 03cb1feb 868c4eae 22776759 7f6a2fa8 7317e652 83514fa3 3bff2378 7f53de45 a4875f3f 5e8c5e7e 50bc28b3 b78215d1 80433cf3 c069ed33 8679f842 322850af 80e077ec bbff12d4 7d157458 d2f722a5 d65238ba f7f2ce31 9f5d983e 5115ebe5 89b1fdd2 52fe98c4 4907b9ef 44eb815a 8ce62db4 8a0842b8 177f908c 3f4a105f 35cd0771 e0bf4b8b e52e4a9e f9e38e5a 584f6568 b7578789 1345b8b2 982d964e cdae286f bd8fbd76 5793f12e 641e45c1 ea06b4d0 724d27d3 d3f5c397 ed415c3c


In [52]:
!cat ../data/sample_submission.csv

id,cell_order
0009d135ece78d,ddfd239c c6cd22db 1372ae9b 90ed07ab 7f388a41 2843a25a 06dbf8cf f9893819 ba55e576 39e937ec e25aa9bd 0a226b6a 8cb8d28a
0010483c12ba9b,54c7cab3 fe66203e 7844d5f8 5ce8863c 4a0777c4 4703bb6d 4a32c095 865ad516 02a0be6d 7f270e34
0010a919d60e4f,aafc3d23 80e077ec b190ebb4 ed415c3c 322850af c069ed33 868c4eae 80433cf3 bd8fbd76 0e2529e8 1345b8b2 cdae286f 4907b9ef d65238ba 641e45c1 7f6a2fa8 982d964e 9f5d983e 22776759 ef01da10 e0bf4b8b 5793f12e 3741e756 bc8eaa53 0115f7f5 177f908c 4356ab34 8679f842 4ae17669 8ce62db4 bac960d3 f9e38e5a ea06b4d0 50bc28b3 a4875f3f 3f4a105f 584f6568 3bff2378 21b6fb8f 7317e652 e52e4a9e bbff12d4 89b1fdd2 f7f2ce31 724d27d3 5e8c5e7e 7d157458 35cd0771 52fe98c4 23607d04 b78215d1 5115ebe5 1d4dbeae b7578789 18ce8cc0 7f53de45 44eb815a d2f722a5 8a0842b8 03cb1feb 83514fa3 d3f5c397
0028856e09c5b7,012c9d02 d22526d1 3ae7ece3 eb293dfc


In [53]:
!kaggle competitions submit -c AI4Code -f ../data/sample_submission.csv -m "Sample submission"

100%|████████████████████████████████████████████| 875/875 [00:02<00:00, 329B/s]
400 - Bad Request
